In [0]:
pip install requests

## Help: https://developers.facebook.com/docs/places/web/search


## List of fields https://developers.facebook.com/docs/graph-api/reference/place-information/

NOTABLE FIELDS: category_list,checkins,engagement,overall_star_rating,price_range,rating_count,restaurant_services(test)

possibly useful: parking,payment_options,restaurant_specialties

<br>
<br>
<br>
https://developers.facebook.com/docs/graph-api/reference/engagement/

basically just likes (the rest are just phrases for showing the likes)
<br>
<br>
<br>

https://developers.facebook.com/docs/graph-api/reference/page-restaurant-specialties/

breakfast coffee dinner drinks lunch
<br>
<br>
<br>

https://developers.facebook.com/docs/graph-api/reference/page-restaurant-services/

restaurant services catering delivery groups kids outdoor pickup reserve takeout waiter walkins
<br>
<br>
<br>
All potential categories: ["ARTS_ENTERTAINMENT", "EDUCATION", "FITNESS_RECREATION", "FOOD_BEVERAGE", "HOTEL_LODGING", "MEDICAL_HEALTH", "SHOPPING_RETAIL", "TRAVEL_TRANSPORTATION"]

# **MUST BE IN ARRAY** AS IN THE FORMAT ABOVE with [ ] and ""


In [0]:
import json

In [0]:
#from google.colab import files
#uploaded= files.upload()
#files.download("outfile.csv")

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My\ Drive/Yelp

/content/drive/My Drive/Yelp


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Yelp')

In [0]:
import pandas as pd
df = pd.read_csv('restaurantbdata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df.head()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.BusinessAcceptsCreditCards,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.ByAppointmentOnly,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,hours,attributes,attributes.DogsAllowed,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Caters,attributes.WheelchairAccessible,attributes.AcceptsInsurance,attributes.RestaurantsTableService,attributes.Ambience,attributes.GoodForMeal,attributes.HappyHour,attributes.BusinessAcceptsBitcoin,attributes.BYOB,attributes.Corkage,attributes.GoodForDancing,attributes.CoatCheck,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours
0,8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"Ethnic Food, Food Trucks, Specialty Food, Impo...",True,False,False,None,False,1,11:30-14:30,11:30-14:30,11:30-14:30,11:30-14:30,11:30-14:30,NaN,NaN,NaN,NaN,NaN,NaN,u'casual',True,u'quiet',False,False,True,'none',True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,0,"Food, Restaurants, Grocery, Middle Eastern",True,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,u'casual',True,NaN,False,True,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,1,"Restaurants, Cheesesteaks, Poutineries",NaN,True,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-21:0,NaN,NaN,False,u'no',NaN,True,NaN,False,NaN,True,u'none',True,False,True,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,1,"Japanese, Fast Food, Food Court, Restaurants",NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,False,NaN,u'none',True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,1,"Persian/Iranian, Turkish, Middle Eastern, Rest...",NaN,NaN,NaN,"{'garage': False, 'street': True, 'validated':...",NaN,NaN,NaN,12:0-21:0,12:0-21:0,12:0-21:0,12:0-0:0,12:0-0:0,11:0-21:0,NaN,NaN,False,u'free',NaN,True,NaN,True,NaN,True,u'full_bar',True,True,True,True,NaN,True,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_toronto = df.loc[df['city'] == 'Toronto'].drop('Unnamed: 0', axis=1).reset_index().drop('index', axis=1)

In [0]:
df_toronto.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.BusinessAcceptsCreditCards,attributes.BikeParking,attributes.GoodForKids,attributes.BusinessParking,attributes.ByAppointmentOnly,attributes.RestaurantsPriceRange2,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,hours,attributes,attributes.DogsAllowed,attributes.WiFi,attributes.RestaurantsAttire,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsReservations,attributes.RestaurantsGoodForGroups,attributes.HasTV,attributes.Alcohol,attributes.RestaurantsDelivery,attributes.OutdoorSeating,attributes.Caters,attributes.WheelchairAccessible,attributes.AcceptsInsurance,attributes.RestaurantsTableService,attributes.Ambience,attributes.GoodForMeal,attributes.HappyHour,attributes.BusinessAcceptsBitcoin,attributes.BYOB,attributes.Corkage,attributes.GoodForDancing,attributes.CoatCheck,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours
0,rVBPQdeayMYht4Uv_FOLHg,Gourmet Burger Company,843 Kipling Avenue,Toronto,ON,M8Z 5G9,43.633291,-79.531768,3.0,13,0,"Restaurants, Burgers, Food",NaN,NaN,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,u'average',False,True,False,u'none',False,False,False,NaN,NaN,NaN,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,ON,M4M 1Y3,43.666376,-79.348773,4.0,116,1,"Vietnamese, Restaurants",NaN,True,True,"{'garage': False, 'street': True, 'validated':...",NaN,1,11:0-22:0,11:0-22:0,12:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,NaN,NaN,NaN,u'no',u'casual',True,u'average',False,True,True,u'beer_and_wine',False,False,False,NaN,NaN,True,"{'romantic': False, 'intimate': False, 'classy...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8k62wYhDVq1-652YbJi5eg,Tim Hortons,90 Adelaide Street W,Toronto,ON,M5H 3V9,43.649859,-79.382060,3.0,8,1,"Bagels, Donuts, Food, Cafes, Coffee & Tea, Res...",NaN,False,True,"{'garage': False, 'street': False, 'validated'...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no','casual',True,NaN,False,True,True,'none',False,False,NaN,NaN,NaN,NaN,"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0DnQh8SE8BSnvJltGCCiWg,Chick-N-Joy,3-1265 York Mills Road,Toronto,ON,M3A 1Z3,43.765279,-79.326248,3.0,11,1,"Fast Food, Restaurants, Chicken Shop",NaN,True,True,"{'garage': False, 'street': False, 'validated'...",NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'no',u'casual',True,'loud',False,True,True,'none',True,True,True,NaN,NaN,False,"{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
4,NLaK58WvlNQdUunSIkt-jA,Zav Coffee Shop & Gallery,2048 Danforth Avenue,Toronto,ON,M4C 1J6,43.685608,-79.313936,4.5,24,1,"Coffee & Tea, Restaurants, Sandwiches, Food",NaN,True,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,1,0:0-0:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,8:30-17:0,8:30-17:0,NaN,NaN,False,u'free',NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_toronto.to_json('restaurant_toronto.json', orient="records", lines=True)

In [0]:
import numpy as np
import requests 
import json
# https://developers.facebook.com/tools/explorer/
URL = "https://graph.facebook.com/search?type=place"

#must choose either center+distance(meters) or q
#center='30.5607,-96.2623'
distance='50'
#q = 'Little Caesar\'s'
#categories = '["ARTS_ENTERTAINMENT", "EDUCATION", "FITNESS_RECREATION", "FOOD_BEVERAGE", "HOTEL_LODGING", "MEDICAL_HEALTH", "SHOPPING_RETAIL", "TRAVEL_TRANSPORTATION"]'
fields='name,category_list,checkins,engagement,overall_star_rating,price_range,rating_count,restaurant_services,parking,payment_options,restaurant_specialties'
# message tony for access token or make a facebook developer account and get one
access_token=''
  
#print(df_toronto['name'][i])
#print(str(df_toronto['latitude'][i])+',' +str(df_toronto['longitude'][i]))
fieldlist = ['category_list','checkins','engagement','overall_star_rating','price_range','rating_count','restaurant_services','parking','payment_options','restaurant_specialties']
entries = []
for i in range(2937, 3000):
  PARAMS = {'q':df_toronto['name'][i],'center':str(df_toronto['latitude'][i])+',' +str(df_toronto['longitude'][i]), 'distance':distance, 'fields':fields, 'access_token':access_token} 
    
  r = requests.get(url = URL, params = PARAMS) 

  print(r.json())
  data = r.json()['data']
  if len(data) == 0:
    data.append({})
  entry = []
  for restaurant in data:
    entry.append(restaurant)
    for field in fieldlist:
      if field not in restaurant.keys():
        if field == 'category_list':
          restaurant[field] = []
        elif field == 'engagement':
          restaurant[field] = {'count': np.NaN}
        elif field == 'restaurant_services':
          restaurant[field] = {'catering': np.NaN, 'delivery': np.NaN, 'groups': np.NaN, 'kids': np.NaN, 'outdoor': np.NaN, 'reserve': np.NaN, 'takeout': np.NaN, 'waiter': np.NaN, 'walkins': np.NaN}
        elif field == 'parking':
          restaurant[field] = {'lot': np.NaN,'street': np.NaN, 'valet': np.NaN}
        elif field == 'payment_options':
          restaurant[field] = {'cash_only': np.NaN, 'discover': np.NaN, 'mastercard': np.NaN, 'visa': np.NaN}
        elif field == 'restaurant_specialties':
          restaurant[field] = {'coffee': np.NaN,'dinner': np.NaN,'drinks': np.NaN,'lunch': np.NaN}
        else:
          restaurant[field] = np.NaN
  with open('facebook_data_toronto.json', 'a', encoding='utf-8') as outfile:
    json.dump(json.dumps(entry), outfile)
    outfile.write('\n')


{'data': [{'name': 'Wafflian - Belgian Waffle, Crepe & Cupcake Bar', 'category_list': [{'id': '128673187201735', 'name': 'Coffee Shop'}, {'id': '325310894151007', 'name': 'Cupcake Shop'}, {'id': '273819889375819', 'name': 'Restaurant'}], 'checkins': 141, 'engagement': {'count': 379, 'social_sentence': '379 people like this.'}, 'overall_star_rating': 4.3, 'price_range': '$', 'rating_count': 23, 'restaurant_services': {'delivery': False, 'catering': True, 'groups': True, 'kids': True, 'outdoor': True, 'reserve': True, 'takeout': True, 'waiter': True, 'walkins': True}, 'parking': {'lot': 1, 'street': 1, 'valet': 0}, 'payment_options': {'amex': 0, 'cash_only': 1, 'discover': 0, 'mastercard': 1, 'visa': 1}, 'restaurant_specialties': {'breakfast': 1, 'coffee': 1, 'dinner': 1, 'drinks': 1, 'lunch': 1}, 'id': '469936633101099'}]}
{'data': []}
{'data': [{'name': 'Aroma Espresso Bar Canada', 'category_list': [{'id': '128673187201735', 'name': 'Coffee Shop'}], 'checkins': 74, 'engagement': {'coun

In [0]:
i

2937

In [0]:
df.shape[0]

63944